In [ ]:
from waymo_open_dataset import dataset_pb2

import json
import os

import numpy as np
import tensorflow as tf
from PIL import Image
from tqdm import tqdm
from waymo_open_dataset import label_pb2
from waymo_open_dataset.protos import camera_segmentation_pb2 as cs_pb2
from waymo_open_dataset.utils import box_utils


import matplotlib.pyplot as plt


import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
from pathlib import Path


WOPD_DATA_ROOT = Path("/media/nvme1/waymo_perception/training")


tfrecords_file_list = list(WOPD_DATA_ROOT.glob("*.tfrecord"))

In [ ]:
import io
from pyquaternion import Quaternion

from d123.geometry import StateSE3
from d123.geometry.bounding_box import BoundingBoxSE3

from waymo_open_dataset.utils import frame_utils


# Frame attributes:
#   context: <class 'waymo_open_dataset.dataset_pb2.Context'>
#   timestamp_micros: <class 'int'>
#   pose: <class 'waymo_open_dataset.dataset_pb2.Transform'>
#   images: List with 5 images
#   lasers: <class 'google._upb._message.RepeatedCompositeContainer'>
#     Length: 5
#   laser_labels: <class 'google._upb._message.RepeatedCompositeContainer'>
#     Length: 0
#   projected_lidar_labels: <class 'google._upb._message.RepeatedCompositeContainer'>
#     Length: 0
#   camera_labels: <class 'google._upb._message.RepeatedCompositeContainer'>
#     Length: 0
#   no_label_zones: <class 'google._upb._message.RepeatedCompositeContainer'>
#     Length: 0
#   map_features: <class 'google._upb._message.RepeatedCompositeContainer'>
#     Length: 0
#   map_pose_offset: <class 'waymo_open_dataset.protos.vector_pb2.Vector3d'>

file_idx = 0
pathname = tfrecords_file_list[file_idx]
dataset = tf.data.TFRecordDataset(pathname, compression_type="")
num_frames = sum(1 for _ in dataset)


def read_jpg_image(data: bytes) -> np.ndarray:
    """Read a JPEG image from bytes and return it as a numpy array."""
    image = Image.open(io.BytesIO(data))
    return np.array(image)


ego_state_se3s = []
front_images = []
dataset = tf.data.TFRecordDataset(pathname, compression_type="")

boxes = []

for frame_idx, data in enumerate(dataset):

    frame = dataset_pb2.Frame()
    frame.ParseFromString(data.numpy())
    print("Frame attributes:")
    for field in frame.DESCRIPTOR.fields:
        field_name = field.name
        if hasattr(frame, field_name):
            value = getattr(frame, field_name)
            if field_name != "images":  # Don't print the whole image data
                print(f"  {field_name}: {type(value)}")
                if hasattr(value, "__len__") and not isinstance(value, (str, bytes)):
                    print(f"    Length: {len(value)}")
            else:
                print(f"  {field_name}: List with {len(value)} images")

    # # plt.show()
    if frame_idx == 0:
        break

ego_state_se3s = np.array(ego_state_se3s, dtype=np.float64)

In [ ]:
frame.context.stats.location

In [ ]:
from collections import defaultdict

from d123.geometry.utils.units import mph_to_mps


dataset = tf.data.TFRecordDataset(pathname, compression_type="")
for frame_idx, data in enumerate(dataset):
    frame = dataset_pb2.Frame()
    frame.ParseFromString(data.numpy())
    break

driveways = {}
stop_signs = {}
crosswalks = {}
road_edges = {}
road_lines = {}

lanes = {}
lanes_type = {}
lanes_speed_limit_mps = {}
lanes_interpolate = {}
lanes_successors = defaultdict(list)
lanes_predecessors = defaultdict(list)

lanes_map_features = {}

for map_feature in frame.map_features:

    if map_feature.HasField("lane"):
        lanes[map_feature.id] = np.array([[p.x, p.y, p.z] for p in map_feature.lane.polyline])
        lanes_map_features[map_feature.id] = map_feature
        lanes_type[map_feature.id] = map_feature.lane.type
        lanes_speed_limit_mps[map_feature.id] = mph_to_mps(map_feature.lane.speed_limit_mph)
        for lane_id_ in map_feature.lane.exit_lanes:
            lanes_successors[map_feature.id].append(lane_id_)
        for lane_id_ in map_feature.lane.exit_lanes:
            lanes_predecessors[map_feature.id].append(lane_id_)

    elif map_feature.HasField("road_line"):
        road_lines[map_feature.id] = np.array([[p.x, p.y, p.z] for p in map_feature.road_line.polyline])
    elif map_feature.HasField("road_edge"):
        road_edges[map_feature.id] = np.array([[p.x, p.y, p.z] for p in map_feature.road_edge.polyline])
    elif map_feature.HasField("stop_sign"):
        pass
    elif map_feature.HasField("crosswalk"):
        outline = np.array([[p.x, p.y, p.z] for p in map_feature.crosswalk.polygon])
        crosswalks[map_feature.id] = outline
    elif map_feature.HasField("speed_bump"):
        pass
    elif map_feature.HasField("driveway"):
        # print(map_feature.driveway)
        outline = np.array([[p.x, p.y, p.z] for p in map_feature.driveway.polygon])
        driveways[map_feature.id] = outline

    # print(f"Roadline: {map_feature.road_line}")

In [ ]:
from d123.common.visualization.matplotlib.utils import add_non_repeating_legend_to_ax


fig, ax = plt.subplots(figsize=(30, 30))

for road_edge in road_edges.values():
    # print(len(driveway))
    ax.plot(road_edge[:, 0], road_edge[:, 1], color="blue", label="road_edge")

for lane in lanes.values():
    # print(len(driveway))
    ax.plot(lane[:, 0], lane[:, 1], color="gray", label="lane")


for road_line in road_lines.values():
    # print(len(driveway))
    ax.plot(road_line[:, 0], road_line[:, 1], color="orange", label="road_line")

for driveway in driveways.values():
    # print(len(driveway))
    ax.plot(driveway[:, 0], driveway[:, 1], color="green", label="driveway")


for crosswalk in crosswalks.values():
    # print(len(driveway))
    ax.plot(crosswalk[:, 0], crosswalk[:, 1], color="violet", label="crosswalk")


add_non_repeating_legend_to_ax(ax)

ax.set_aspect("equal")

In [ ]:
from d123.dataset.dataset_specific.wopd.wopd_map_utils import extract_lane_boundaries


left_boundaries, right_boundaries = extract_lane_boundaries(
    lanes=lanes,
    lanes_successors=lanes_successors,
    lanes_predecessors=lanes_predecessors,
    road_lines=road_lines,
    road_edges=road_edges,
)

In [ ]:
size = 20
fig, ax = plt.subplots(figsize=(size, size))

for lane_id, lane_ in lanes.items():
    if lane_id not in left_boundaries or lane_id not in right_boundaries:
        continue
    left_boundary = left_boundaries[lane_id].array
    right_boundary = right_boundaries[lane_id].array

    assert len(left_boundary) > 0 and len(right_boundary) > 0
    ax.plot(left_boundary[:, 0], left_boundary[:, 1], color="lime")
    ax.plot(right_boundary[:, 0], right_boundary[:, 1], color="red")

    ax.plot(lane_[:, 0], lane_[:, 1], color="black")



ax.set_aspect("equal")

In [ ]:
# 

In [ ]:
lane_idx = 158

lane = lanes_map_features[lane_idx].lane
lane_id = lanes_map_features[lane_idx].id


import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 10))

ax.plot(lanes[lane_id][:, 0], lanes[lane_id][:, 1], color="black", label="Lane Centerline", alpha=0.5)

for left_boundary in lane.left_boundaries:
    print("left", left_boundary)
    try:
        boundary = road_lines[left_boundary.boundary_feature_id]

    except KeyError:
        print(f"Boundary feature ID {left_boundary.boundary_feature_id} not found.")
        boundary = road_edges[left_boundary.boundary_feature_id]
    ax.plot(
        boundary[:, 0],
        boundary[:, 1],
        color="lime",
        linestyle="--",
        label="Left Neighbor",
    )
    ax.plot(
        lanes[lane_id][left_boundary.lane_start_index : left_boundary.lane_end_index, 0],
        lanes[lane_id][left_boundary.lane_start_index : left_boundary.lane_end_index, 1],
        color="lime",
        linestyle="-",
        label="Left Neighbor",
    )


for right_boundary in lane.right_boundaries:
    print("right", right_boundary)
    try:
        boundary = road_lines[right_boundary.boundary_feature_id]

    except KeyError:
        print(f"Boundary feature ID {right_boundary.boundary_feature_id} not found.")
        boundary = road_edges[right_boundary.boundary_feature_id]

    ax.plot(
        boundary[:, 0],
        boundary[:, 1],
        color="red",
        linestyle="--",
        label="Right Neighbor",
    )
    ax.plot(
        lanes[lane_id][right_boundary.lane_start_index : right_boundary.lane_end_index, 0],
        lanes[lane_id][right_boundary.lane_start_index : right_boundary.lane_end_index, 1],
        color="red",
        linestyle="-",
        label="Right Neighbor",
    )


ax.set_aspect("equal")
# lanes

zoom_out = 10
lane = lanes[lane_id]
ax.set_xlim(lane[:, 0].min() - zoom_out, lane[:, 0].max() + zoom_out)
ax.set_ylim(lane[:, 1].min() - zoom_out, lane[:, 1].max() + zoom_out)

In [ ]:
from d123.geometry.polyline import Polyline3D
import numpy as np


polyline_3d = Polyline3D.from_array(lanes[0])


# Create left and right boundaries for the lane


def create_lane_boundaries(polyline_3d, width=2.0):
    """
    Create left and right boundaries for a lane by offsetting perpendicular to the heading.

    Args:
        polyline_3d: Polyline3D object representing the centerline
        width: Total width of the lane in meters

    Returns:
        left_boundary: numpy array representing the left boundary
        right_boundary: numpy array representing the right boundary
    """
    points = polyline_3d.array
    half_width = width / 2.0

    # Calculate the direction vectors between consecutive points
    directions = np.diff(points, axis=0)

    # Normalize the direction vectors
    directions_norm = np.linalg.norm(directions, axis=1, keepdims=True)
    directions_normalized = directions / directions_norm

    # Calculate perpendicular vectors in the xy plane (z remains 0)
    perpendiculars = np.zeros_like(directions)
    perpendiculars[:, 0] = -directions_normalized[:, 1]  # -dy
    perpendiculars[:, 1] = directions_normalized[:, 0]  # dx

    # Create boundaries (need to handle the last point separately)
    left_boundary = points[:-1] + perpendiculars * half_width
    right_boundary = points[:-1] - perpendiculars * half_width

    # Handle the last point based on the last direction
    last_perp = perpendiculars[-1]
    left_boundary = np.vstack([left_boundary, points[-1] + last_perp * half_width])
    right_boundary = np.vstack([right_boundary, points[-1] - last_perp * half_width])

    return left_boundary, right_boundary


# Create the boundaries with a 4m wide lane
left_boundary, right_boundary = create_lane_boundaries(polyline_3d, width=4.0)

# Plot the centerline and the boundaries
plt.figure(figsize=(12, 8))
plt.plot(polyline_3d.array[:, 0], polyline_3d.array[:, 1], "g-", label="Centerline")
plt.plot(left_boundary[:, 0], left_boundary[:, 1], "b-", label="Left Boundary")
plt.plot(right_boundary[:, 0], right_boundary[:, 1], "r-", label="Right Boundary")
plt.axis("equal")
plt.grid(True)
plt.legend()
plt.title("Lane with Left and Right Boundaries")

In [ ]:
lane_idx = 59

lane = lanes_map_features[lane_idx].lane
lane_id = lanes_map_features[lane_idx].id


# for right_neighbor in lane.right_neighbors:
# print(right_neighbor)

# lane

import matplotlib.pyplot as plt
from torch import le

fig, ax = plt.subplots(figsize=(10, 10))

ax.plot(lanes[lane_id][:, 0], lanes[lane_id][:, 1], color="blue", label="Lane Centerline")

for right_neighbor in lane.right_neighbors:
    # print("right", right_neighbor)
    ax.plot(
        lanes[right_neighbor.feature_id][:, 0],
        lanes[right_neighbor.feature_id][:, 1],
        color="red",
        linestyle="--",
        label="Right Neighbor",
    )
    ax.plot(
        lanes[right_neighbor.feature_id][right_neighbor.neighbor_start_index: right_neighbor.neighbor_end_index, 0],
        lanes[right_neighbor.feature_id][right_neighbor.neighbor_start_index: right_neighbor.neighbor_end_index, 1],
        color="red",
        label="Right Neighbor",
    )


for left_neighbor in lane.left_neighbors:
    # print("left", left_neighbor)
    ax.plot(
        lanes[left_neighbor.feature_id][:, 0],
        lanes[left_neighbor.feature_id][:, 1],
        color="lime",
        linestyle="--",
        label="Left Neighbor",
    )
    ax.plot(
        lanes[left_neighbor.feature_id][left_neighbor.neighbor_start_index: left_neighbor.neighbor_end_index, 0],
        lanes[left_neighbor.feature_id][left_neighbor.neighbor_start_index: left_neighbor.neighbor_end_index, 1],
        color="lime",
        label="Left Neighbor",
    )


ax.set_aspect("equal")
# lanes

In [ ]:
lane_idx = 48

lane = lanes_map_features[lane_idx].lane
lane_id = lanes_map_features[lane_idx].id




import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 10))

ax.scatter(lanes[lane_id][0, 0], lanes[lane_id][0, 1], color="black", label="Lane Centerline", alpha=0.5)
ax.scatter(lanes[lane_id][-1, 0], lanes[lane_id][-1, 1], color="blue", label="Lane Centerline", alpha=0.5)

for entry_lane in lane.entry_lanes:
    # print("right", right_neighbor)
    ax.plot(
        lanes[entry_lane][:, 0],
        lanes[entry_lane][:, 1],
        color="red",
        linestyle="--",
        label="Right Neighbor",
    )


for exit_lane in lane.exit_lanes:
    # print("left", left_neighbor)
    ax.plot(
        lanes[exit_lane][:, 0],
        lanes[exit_lane][:, 1],
        color="lime",
        linestyle="--",
        label="Left Neighbor",
    )


ax.set_aspect("equal")
# lanes

In [ ]:
lane_idx = 196

lane = lanes_map_features[lane_idx].lane
lane_id = lanes_map_features[lane_idx].id


import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 10))

ax.plot(lanes[lane_id][:, 0], lanes[lane_id][:, 1], color="black", label="Lane Centerline", alpha=0.5)

for left_boundary in lane.left_boundaries:
    print("left", left_boundary)
    try:
        boundary = road_lines[left_boundary.boundary_feature_id]

    except KeyError:
        print(f"Boundary feature ID {left_boundary.boundary_feature_id} not found.")
        boundary = road_edges[left_boundary.boundary_feature_id]
    ax.plot(
        boundary[:, 0],
        boundary[:, 1],
        color="lime",
        linestyle="--",
        label="Left Neighbor",
    )
    ax.plot(
        lanes[lane_id][left_boundary.lane_start_index : left_boundary.lane_end_index, 0],
        lanes[lane_id][left_boundary.lane_start_index : left_boundary.lane_end_index, 1],
        color="lime",
        linestyle="-",
        label="Left Neighbor",
    )


for right_boundary in lane.right_boundaries:
    print("right", right_boundary)
    try:
        boundary = road_lines[right_boundary.boundary_feature_id]

    except KeyError:
        print(f"Boundary feature ID {right_boundary.boundary_feature_id} not found.")
        boundary = road_edges[right_boundary.boundary_feature_id]

    ax.plot(
        boundary[:, 0],
        boundary[:, 1],
        color="red",
        linestyle="--",
        label="Right Neighbor",
    )
    ax.plot(
        lanes[lane_id][right_boundary.lane_start_index : right_boundary.lane_end_index, 0],
        lanes[lane_id][right_boundary.lane_start_index : right_boundary.lane_end_index, 1],
        color="red",
        linestyle="-",
        label="Right Neighbor",
    )


ax.set_aspect("equal")
# lanes